<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>LSTM - Clase 5</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt


In [7]:
# Lee el archivo como un DataFrame
data  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(data.head())
print(df_pid_validos.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [8]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

In [9]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
# En un nuevo Data Frame llamado "DATA"
data = data[data['product_id'].isin(product_ids_validos)]

In [ ]:
# Filtrar los datos para el período especificado
data['periodo'] = data['periodo'].astype(str)
data = data[(data['periodo'] >= '201801') & (data['periodo'] <= '201812')]

# Agrupar por product_id
products = data['product_id'].unique()

# Función para crear las secuencias de datos
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Diccionario para almacenar los resultados
results = {'product_id': [], 'tn': []}

for product_id in products:
    product_data = data[data['product_id'] == product_id]['tn'].values.reshape(-1, 1)
    
    # Estandarizar los datos
    scaler = StandardScaler()
    product_data = scaler.fit_transform(product_data)
    
    # Crear secuencias de datos
    seq_length = 12
    X, y = create_sequences(product_data, seq_length)
    
    # Dividir los datos en entrenamiento y prueba
    train_size = int(len(X) * 0.8)
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:], y[train_size:]
    
    # Crear el modelo LSTM
    model = Sequential()
    model.add(Input(shape=(seq_length, 1)))  # Utilizar Input en lugar de input_shape
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1)
    
    # Realizar predicciones
    predictions = model.predict(X_test)
    
    # Revertir la estandarización
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test)
    
    # Almacenar los resultados
    results['product_id'].append(product_id)
    results['tn'].append(predictions[-1][0])

# Convertir los resultados a un DataFrame y guardarlos en un CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_tn.v2.csv', index=False)

Epoch 1/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.6393
Epoch 2/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.6933
Epoch 3/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9514
Epoch 4/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8200
Epoch 5/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.7759
Epoch 6/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9528
Epoch 7/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7461
Epoch 8/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6424
Epoch 9/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9876
Epoch 10/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7510
Epoch 11/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.6736
Epoch 12/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9328
Epoch 13/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8740
Epoch 14/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.7988
Epoch 15/50
659/659 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - lo

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 